# 1. Descrição do problema de negócio
..

# 2. Carregando dados

## 2.1 Importando dependências

Importando as bibliotecas necessárias para as etapas inicias de carregamento de dados, exploração e transformação.

In [8]:
## Load dependencies.
import pandas as pd
import json
import requests
import numpy as np

## 2.1 Carregando dados através da API CKAN.

A base de dados carregada aqui é referente as operações de financiamento não automáticas dos dados abertos do BNDES.

In [3]:
## Define resource identifier and server to create our URL string for an API request.
res_id = '332d446e-d340-46ef-af64-ee6f36e7bd50' ## Operações de financiamento não automáticas.
server = 'https://dadosabertos.bndes.gov.br'
query = (f"{server}/api/3/action/datastore_search?resource_id={res_id}")

print('Url para consulta:', query)


Url para consulta: https://dadosabertos.bndes.gov.br/api/3/action/datastore_search?resource_id=332d446e-d340-46ef-af64-ee6f36e7bd50


In [4]:
## Get number of observations of the dataset.
try:
    data = requests.get(query)
    if data.json()['success'] == True:
        limit = data.json()['result']['total']

    else:
        print(data.json()['error'])
except:
    data.raise_for_status()


In [5]:
## Use limit to request a single batch for the full dataset.
big_query = (f"{query}&limit={limit}")
try:
    data = requests.get(big_query)
    if data.json()['success'] == True:
        df_fin = pd.DataFrame(data.json()['result']['records'])

    else:
        print(data.json()['error'])
except:
    data.raise_for_status()


# 3. Data wrangling - Transformando e mapeando os dados para análise.


## 3.1 Explorando o dataset 'Operações não automáticas'

In [5]:
## Return variables definition.
data.json()['result']['fields']


[{'type': 'int', 'id': '_id'},
 {'type': 'text', 'id': 'cliente'},
 {'type': 'text', 'id': 'cnpj'},
 {'type': 'text', 'id': 'descricao_do_projeto'},
 {'type': 'text', 'id': 'uf'},
 {'type': 'text', 'id': 'municipio'},
 {'type': 'numeric', 'id': 'municipio_codigo'},
 {'type': 'numeric', 'id': 'numero_do_contrato'},
 {'type': 'timestamp', 'id': 'data_da_contratacao'},
 {'type': 'numeric', 'id': 'valor_contratado_reais'},
 {'type': 'numeric', 'id': 'valor_desembolsado_reais'},
 {'type': 'text', 'id': 'fonte_de_recurso_desembolsos'},
 {'type': 'text', 'id': 'custo_financeiro'},
 {'type': 'numeric', 'id': 'juros'},
 {'type': 'numeric', 'id': 'prazo_carencia_meses'},
 {'type': 'numeric', 'id': 'prazo_amortizacao_meses'},
 {'type': 'text', 'id': 'modalidade_de_apoio'},
 {'type': 'text', 'id': 'forma_de_apoio'},
 {'type': 'text', 'id': 'produto'},
 {'type': 'text', 'id': 'instrumento_financeiro'},
 {'type': 'text', 'id': 'inovacao'},
 {'type': 'text', 'id': 'area_operacional'},
 {'type': 'text

In [6]:
## Overview of the dataset.
df_fin.head()

,_id,cliente,cnpj,descricao_do_projeto,uf,municipio,municipio_codigo,numero_do_contrato,data_da_contratacao,valor_contratado_reais,...,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_da_instituicao_financeira_credenciada,tipo_de_garantia,tipo_de_excepcionalidade,situacao_do_contrato
0,1,MUNICIPIO DE RIBEIRAO PRETO,56.024.581/0001-56,PROGRAMA DE MODERNIZACAO DA ADMINISTRACAO TRIB...,SP,RIBEIRAO PRETO,3543402,1247021,2002-01-02T00:00:00,9.090000e+06,...,ADMINISTRACAO PUBLICA EM GERAL ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,ADMINISTRAÇÃO PÚBLICA DIRETA - GOVERNO MUNICIPAL,----------,----------,"OUTRA, DE NATUREZA ESPECÍFICA OU MISTA",2021-07-29T00:00:00,LIQUIDADO
1,2,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03T00:00:00,7.066000e+05,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,2021-07-29T00:00:00,LIQUIDADO
2,3,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03T00:00:00,7.569159e+04,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,2021-07-29T00:00:00,LIQUIDADO
3,4,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03T00:00:00,6.039814e+05,...,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,2021-07-29T00:00:00,LIQUIDADO
4,39,ELETROPAULO METROPOLITANA ELETRICIDADE DE SAO ...,61.695.227/0001-93,SUPRIR PARTE DAS INSUFICIENCIAS DE RECURSOS DA...,SP,SEM MUNICÍPIO,9999999,2204531,2002-02-04T00:00:00,1.820297e+08,...,DISTRIBUICAO DE ENERGIA ELETRICA ...,INFRAESTRUTURA,ENERGIA ELÉTRICA,GRANDE,PRIVADA,----------,----------,REAL,2021-07-29T00:00:00,LIQUIDADO


In [7]:
## Dataset dimensions
df_fin.shape

(19565, 35)

In [8]:
## Veryfing duplicated records
df_fin.duplicated().sum()

0

In [9]:
## Veryfing NA's
df_fin.isna().sum()

_id                                           0
cliente                                       0
cnpj                                          0
descricao_do_projeto                          0
uf                                            0
municipio                                     0
municipio_codigo                              0
numero_do_contrato                            0
data_da_contratacao                           0
valor_contratado_reais                        0
valor_desembolsado_reais                      0
fonte_de_recurso_desembolsos                  0
custo_financeiro                              0
juros                                         0
prazo_carencia_meses                          0
prazo_amortizacao_meses                       0
modalidade_de_apoio                           0
forma_de_apoio                                0
produto                                       0
instrumento_financeiro                        0
inovacao                                

## 3.2 Explorando os setores para entender o pareamento necessários dos dados SEEG

In [31]:
print(df_fin["setor_bndes"].unique())


['COMERCIO/SERVICOS' 'INFRAESTRUTURA' 'INDUSTRIA' 'AGROPECUÁRIA']


In [26]:
index = np.where(df_fin['setor_bndes']=='AGROPECUÁRIA')

IndexError: tuple index out of range

In [30]:
df_fin.loc[index,'subsetor_bndes']

29       AGROPECUÁRIA
30       AGROPECUÁRIA
31       AGROPECUÁRIA
32       AGROPECUÁRIA
34       AGROPECUÁRIA
             ...     
19542    AGROPECUÁRIA
19543    AGROPECUÁRIA
19544    AGROPECUÁRIA
19550    AGROPECUÁRIA
19555    AGROPECUÁRIA
Name: subsetor_bndes, Length: 729, dtype: object